**This notebook has implementation and performance analysis of common algorithms for solving the vehicle routing problem on Augerat-1995 dataset set P.**

---


**Structure:**

1. Imports
2. Implementation:

*   Neirest Neighbor
*   LKH-3
*   Simulated Annealing

3. Solution Routes


---

Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import time
import glob
import os
import re
import random

from itertools import combinations
from scipy.spatial.distance import pdist, squareform

Data Loading

In [2]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/DNA-Insipred NNs/Input/augerat-1995-set-p.zip'

Archive:  /content/drive/MyDrive/DNA-Insipred NNs/Input/augerat-1995-set-p.zip
  inflating: readme.txt              
  inflating: P-n016-k08.xml          
  inflating: P-n019-k02.xml          
  inflating: P-n020-k02.xml          
  inflating: P-n021-k02.xml          
  inflating: P-n022-k02.xml          
  inflating: P-n022-k08.xml          
  inflating: P-n023-k08.xml          
  inflating: P-n040-k05.xml          
  inflating: P-n045-k05.xml          
  inflating: P-n050-k07.xml          
  inflating: P-n050-k08.xml          
  inflating: P-n050-k10.xml          
  inflating: P-n051-k10.xml          
  inflating: P-n055-k07.xml          
  inflating: P-n055-k08.xml          
  inflating: P-n055-k10.xml          
  inflating: P-n055-k15.xml          
  inflating: P-n060-k10.xml          
  inflating: P-n060-k15.xml          
  inflating: P-n065-k10.xml          
  inflating: P-n070-k10.xml          
  inflating: P-n076-k04.xml          
  inflating: P-n076-k05.xml          
  inflati

Implementation: Neirest Neighbor



In [4]:
def parse_xml_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Parse nodes
    nodes = []
    for node in root.findall('.//node'):
        node_id = int(node.get('id'))
        node_type = int(node.get('type'))
        cx = float(node.find('cx').text)
        cy = float(node.find('cy').text)
        nodes.append((node_id, node_type, cx, cy))

    # Parse vehicle capacity
    capacity = float(root.find('.//vehicle_profile/capacity').text)

    # Extract number of vehicles from filename
    filename = os.path.basename(file_path)
    match = re.search(r'-k(\d+)', filename)
    num_vehicles = int(match.group(1)) if match else len(root.findall('.//vehicle_profile'))

    # Parse demands
    demands = [0]  # Depot has no demand
    for request in root.findall('.//request'):
        demands.append(float(request.find('quantity').text))

    return nodes, capacity, num_vehicles, demands

def create_distance_matrix(nodes):
    coordinates = np.array([(x, y) for _, _, x, y in nodes])
    return squareform(pdist(coordinates))

def nearest_neighbor_vrp(num_cities, num_vehicles, capacity, demands):
    unvisited = set(range(1, num_cities))  # Skip depot (0)
    routes = []

    for _ in range(num_vehicles):
        if not unvisited:
            break

        current_route = []
        current_capacity = capacity
        current_city = 0  # Start from depot

        while unvisited:
            # Find nearest unvisited city that fits capacity
            nearest_city = None
            min_distance = float('inf')

            for city in unvisited:
                if demands[city] <= current_capacity:
                    distance = dist_matrix[current_city, city]
                    if distance < min_distance:
                        min_distance = distance
                        nearest_city = city

            if nearest_city is None:
                break

            current_route.append(nearest_city)
            current_capacity -= demands[nearest_city]
            current_city = nearest_city
            unvisited.remove(nearest_city)

        if current_route:
            routes.append(tuple(current_route))

    return routes

def calculate_route_distance(route):
    distance = dist_matrix[0, route[0]]  # Depot to first city
    for i in range(len(route)-1):
        distance += dist_matrix[route[i], route[i+1]]
    distance += dist_matrix[route[-1], 0]  # Last city to depot
    return distance

def solve_vrp(file_path):
    # Parse XML data
    nodes, capacity, num_vehicles, demands = parse_xml_file(file_path)
    num_cities = len(nodes)

    # Create distance matrix
    global dist_matrix
    dist_matrix = create_distance_matrix(nodes)

    print(f"\nSolving {os.path.basename(file_path)}")
    print(f"Number of cities: {num_cities}")
    print(f"Number of vehicles: {num_vehicles}")
    print(f"Vehicle capacity: {capacity}")

    # Start timer
    start_time = time.time()

    # Run nearest neighbor algorithm
    routes = nearest_neighbor_vrp(num_cities, num_vehicles, capacity, demands)

    # Print route details
    print("\nSolution routes:")
    total_distance = 0
    for i, route in enumerate(routes, 1):
        route_distance = calculate_route_distance(route)
        total_distance += route_distance
        print(f"Route {i}: {[0] + list(route) + [0]} (Distance: {route_distance:.2f})")
    print(f"Total distance: {total_distance:.2f}")

    # Print execution time
    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.2f} seconds")


def main():
    # Get all XML files
    xml_files = glob.glob('P-n*.xml')

    for file_path in sorted(xml_files):
        solve_vrp(file_path)

if __name__ == "__main__":
    main()


Solving P-n016-k08.xml
Number of cities: 16
Number of vehicles: 8
Vehicle capacity: 35.0

Solution routes:
Route 1: [0, 6, 0] (Distance: 24.08)
Route 2: [0, 1, 10, 13, 0] (Distance: 68.44)
Route 3: [0, 2, 0] (Distance: 42.05)
Route 4: [0, 4, 11, 0] (Distance: 57.39)
Route 5: [0, 7, 5, 9, 0] (Distance: 75.54)
Route 6: [0, 15, 12, 0] (Distance: 66.12)
Route 7: [0, 14, 3, 0] (Distance: 100.91)
Route 8: [0, 8, 0] (Distance: 64.90)
Total distance: 499.44
Execution time: 0.00 seconds

Solving P-n019-k02.xml
Number of cities: 19
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 6, 18, 5, 7, 2, 9, 15, 16, 0] (Distance: 129.15)
Route 2: [0, 1, 10, 12, 14, 4, 11, 3, 8, 17, 13, 0] (Distance: 172.59)
Total distance: 301.75
Execution time: 0.00 seconds

Solving P-n020-k02.xml
Number of cities: 20
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 6, 19, 5, 7, 2, 13, 9, 16, 17, 0] (Distance: 137.32)
Route 2: [0, 1, 10, 12, 15, 4, 11, 3, 8, 18, 14

Implementation: LKH-3

In [5]:
class LKH_VRP:
    def __init__(self, dist_matrix, num_vehicles, capacity, demands):
        self.dist_matrix = dist_matrix
        self.num_cities = len(dist_matrix)
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.demands = demands

    def initial_solution(self):
        unvisited = set(range(1, self.num_cities))
        routes = []

        for _ in range(self.num_vehicles):
            if not unvisited:
                break

            current_route = []
            current_capacity = self.capacity
            current_city = 0

            while unvisited:
                nearest_city = None
                min_distance = float('inf')

                for city in unvisited:
                    if self.demands[city] <= current_capacity:
                        distance = self.dist_matrix[current_city][city]
                        if distance < min_distance:
                            min_distance = distance
                            nearest_city = city

                if nearest_city is None:
                    break

                current_route.append(nearest_city)
                current_capacity -= self.demands[nearest_city]
                current_city = nearest_city
                unvisited.remove(nearest_city)

            if current_route:
                routes.append(current_route)

        return routes

    def calculate_route_length(self, route):
        length = self.dist_matrix[0][route[0]]
        for i in range(len(route) - 1):
            length += self.dist_matrix[route[i]][route[i + 1]]
        length += self.dist_matrix[route[-1]][0]
        return length

    def calculate_total_distance(self, routes):
        return sum(self.calculate_route_length(route) for route in routes)

    def is_feasible_route(self, route):
        return sum(self.demands[i] for i in route) <= self.capacity

    def two_opt_move(self, route, i, j):
        new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
        return new_route

    def three_opt_move(self, route):
        if len(route) < 3:
            return route

        best_route = route
        best_length = self.calculate_route_length(route)

        for i, j, k in combinations(range(len(route)), 3):
            if j <= i + 1 or k <= j + 1:
                continue

            # Try all possible 3-opt combinations
            segments = [route[:i+1], route[i+1:j+1], route[j+1:k+1], route[k+1:]]
            for flip1 in [False, True]:
                for flip2 in [False, True]:
                    for flip3 in [False, True]:
                        if flip1:
                            segments[1] = segments[1][::-1]
                        if flip2:
                            segments[2] = segments[2][::-1]
                        if flip3:
                            segments[3] = segments[3][::-1]

                        new_route = segments[0] + segments[1] + segments[2] + segments[3]
                        new_length = self.calculate_route_length(new_route)

                        if new_length < best_length and self.is_feasible_route(new_route):
                            best_route = new_route
                            best_length = new_length

        return best_route

    def optimize_route(self, route):
        improved = True
        best_route = route

        while improved:
            improved = False
            current_length = self.calculate_route_length(best_route)

            # Apply 2-opt
            for i in range(len(route) - 1):
                for j in range(i + 1, len(route)):
                    new_route = self.two_opt_move(best_route, i, j)
                    if (self.calculate_route_length(new_route) < current_length and
                        self.is_feasible_route(new_route)):
                        best_route = new_route
                        improved = True
                        break
                if improved:
                    break

            # Apply 3-opt
            if not improved:
                new_route = self.three_opt_move(best_route)
                if (self.calculate_route_length(new_route) < current_length and
                    self.is_feasible_route(new_route)):
                    best_route = new_route
                    improved = True

        return best_route

    def solve(self, max_iterations=100):
        best_solution = self.initial_solution()
        best_distance = self.calculate_total_distance(best_solution)

        for _ in range(max_iterations):
            current_solution = [self.optimize_route(route) for route in best_solution]
            current_distance = self.calculate_total_distance(current_solution)

            if current_distance < best_distance:
                best_solution = current_solution
                best_distance = current_distance

        return best_solution, best_distance

def solve_vrp_lkh(file_path):
    # Parse XML data
    nodes, capacity, num_vehicles, demands = parse_xml_file(file_path)

    print(f"\nSolving {os.path.basename(file_path)} with LKH")
    print(f"Number of cities: {len(nodes)}")
    print(f"Number of vehicles: {num_vehicles}")
    print(f"Vehicle capacity: {capacity}")

    # Create distance matrix
    dist_matrix = create_distance_matrix(nodes)

    # Start timer
    start_time = time.time()

    # Create and run LKH solver
    solver = LKH_VRP(dist_matrix, num_vehicles, capacity, demands)
    routes, total_distance = solver.solve()

    # Print route details
    print("\nSolution routes:")
    for i, route in enumerate(routes, 1):
        route_distance = solver.calculate_route_length(route)
        print(f"Route {i}: {[0] + list(route) + [0]} (Distance: {route_distance:.2f})")
    print(f"Total distance: {total_distance:.2f}")

    # Print execution time
    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.2f} seconds")

def main():
    # Get all XML files
    xml_files = glob.glob('P-n*.xml')

    for file_path in sorted(xml_files):
        solve_vrp_lkh(file_path)

if __name__ == "__main__":
    main()


Solving P-n016-k08.xml with LKH
Number of cities: 16
Number of vehicles: 8
Vehicle capacity: 35.0

Solution routes:
Route 1: [0, 6, 0] (Distance: 24.08)
Route 2: [0, 1, 10, 13, 0] (Distance: 68.44)
Route 3: [0, 2, 0] (Distance: 42.05)
Route 4: [0, 4, 11, 0] (Distance: 57.39)
Route 5: [0, 5, 9, 7, 0] (Distance: 68.61)
Route 6: [0, 15, 12, 0] (Distance: 66.12)
Route 7: [0, 14, 3, 0] (Distance: 100.91)
Route 8: [0, 8, 0] (Distance: 64.90)
Total distance: 492.52
Execution time: 0.01 seconds

Solving P-n019-k02.xml with LKH
Number of cities: 19
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 6, 2, 7, 16, 9, 15, 5, 18, 0] (Distance: 109.24)
Route 2: [0, 1, 4, 11, 14, 12, 10, 3, 17, 8, 13, 0] (Distance: 157.90)
Total distance: 267.14
Execution time: 0.24 seconds

Solving P-n020-k02.xml with LKH
Number of cities: 20
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 19, 7, 5, 16, 9, 13, 17, 2, 6, 0] (Distance: 115.60)
Route 2: [0, 1, 4, 1

Implementation: Simulated Annealing

In [6]:
class SimulatedAnnealingVRP:
    def __init__(self, dist_matrix, num_vehicles, capacity, demands):
        self.dist_matrix = dist_matrix
        self.num_cities = len(dist_matrix)
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.demands = demands

    def initial_solution(self):
        unvisited = set(range(1, self.num_cities))
        routes = []

        for _ in range(self.num_vehicles):
            if not unvisited:
                break

            current_route = []
            current_capacity = self.capacity
            current_city = 0

            while unvisited:
                nearest_city = None
                min_distance = float('inf')

                for city in unvisited:
                    if self.demands[city] <= current_capacity:
                        distance = self.dist_matrix[current_city][city]
                        if distance < min_distance:
                            min_distance = distance
                            nearest_city = city

                if nearest_city is None:
                    break

                current_route.append(nearest_city)
                current_capacity -= self.demands[nearest_city]
                current_city = nearest_city
                unvisited.remove(nearest_city)

            if current_route:
                routes.append(current_route)

        return routes

    def calculate_total_distance(self, routes):
        total_distance = 0
        for route in routes:
            if not route:  # Skip empty routes
                continue
            route_distance = self.dist_matrix[0][route[0]]
            for i in range(len(route) - 1):
                route_distance += self.dist_matrix[route[i]][route[i + 1]]
            route_distance += self.dist_matrix[route[-1]][0]
            total_distance += route_distance
        return total_distance

    def is_feasible(self, routes):
        for route in routes:
            if sum(self.demands[i] for i in route) > self.capacity:
                return False
        return True

    def generate_neighbor(self, routes):
        neighbor_routes = [list(route) for route in routes]

        if not neighbor_routes:
            return neighbor_routes

        r1 = random.randint(0, len(neighbor_routes) - 1)
        r2 = random.randint(0, len(neighbor_routes) - 1)

        if len(neighbor_routes[r1]) > 0:
            i = random.randint(0, len(neighbor_routes[r1])-1)
            if len(neighbor_routes[r2]) > 0:
                j = random.randint(0, len(neighbor_routes[r2])-1)
                if r1 == r2:
                    neighbor_routes[r1][i], neighbor_routes[r2][j] = neighbor_routes[r2][j], neighbor_routes[r1][i]
                else:
                    city = neighbor_routes[r1].pop(i)
                    neighbor_routes[r2].insert(j, city)

        neighbor_routes = [route for route in neighbor_routes if route]
        return neighbor_routes

    def solve(self, initial_temperature=1000, cooling_rate=0.99, max_iterations=1000):
        current_solution = self.initial_solution()
        current_cost = self.calculate_total_distance(current_solution)
        best_solution = current_solution.copy()
        best_cost = current_cost
        temperature = initial_temperature

        for _ in range(max_iterations):
            neighbor = self.generate_neighbor(current_solution)

            if neighbor and self.is_feasible(neighbor):
                neighbor_cost = self.calculate_total_distance(neighbor)
                cost_diff = neighbor_cost - current_cost

                if cost_diff < 0 or random.random() < np.exp(-cost_diff / temperature):
                    current_solution = neighbor
                    current_cost = neighbor_cost

                    if current_cost < best_cost:
                        best_solution = current_solution.copy()
                        best_cost = current_cost

            temperature *= cooling_rate
            if temperature < 0.1:
                break

        return best_solution, best_cost

def solve_vrp_sa(file_path):
    nodes, capacity, num_vehicles, demands = parse_xml_file(file_path)

    print(f"\nSolving {os.path.basename(file_path)} with Simulated Annealing")
    print(f"Number of cities: {len(nodes)}")
    print(f"Number of vehicles: {num_vehicles}")
    print(f"Vehicle capacity: {capacity}")

    dist_matrix = create_distance_matrix(nodes)

    start_time = time.time()

    solver = SimulatedAnnealingVRP(dist_matrix, num_vehicles, capacity, demands)
    routes, total_distance = solver.solve()

    print("\nSolution routes:")
    for i, route in enumerate(routes, 1):
        print(f"Route {i}: {[0] + route + [0]}")
    print(f"Total distance: {total_distance:.2f}")

    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time:.2f} seconds")

def main():
    xml_files = glob.glob('P-n*.xml')

    for file_path in sorted(xml_files):
        solve_vrp_sa(file_path)

if __name__ == "__main__":
    main()


Solving P-n016-k08.xml with Simulated Annealing
Number of cities: 16
Number of vehicles: 8
Vehicle capacity: 35.0

Solution routes:
Route 1: [0, 6, 0]
Route 2: [0, 12, 1, 0]
Route 3: [0, 2, 0]
Route 4: [0, 4, 11, 0]
Route 5: [0, 3, 9, 5, 0]
Route 6: [0, 15, 10, 7, 0]
Route 7: [0, 14, 0]
Route 8: [0, 13, 8, 0]
Total distance: 493.89
Execution time: 0.01 seconds

Solving P-n019-k02.xml with Simulated Annealing
Number of cities: 19
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 6, 18, 7, 2, 16, 9, 15, 5, 0]
Route 2: [0, 13, 10, 8, 17, 3, 12, 14, 11, 4, 1, 0]
Total distance: 280.71
Execution time: 0.01 seconds

Solving P-n020-k02.xml with Simulated Annealing
Number of cities: 20
Number of vehicles: 2
Vehicle capacity: 160.0

Solution routes:
Route 1: [0, 7, 2, 17, 9, 16, 5, 19, 6, 0]
Route 2: [0, 10, 3, 18, 8, 13, 14, 12, 15, 11, 4, 1, 0]
Total distance: 295.29
Execution time: 0.01 seconds

Solving P-n021-k02.xml with Simulated Annealing
Number of cities: 21
